In [1]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)
options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)



load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

Warning message:
"package 'dplyr' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'caret' was built under R version 3.6.3"
Loading required package: lattice

Warning message:
"package 'lattice' was built under R version 3.6.2"
Loading required package: ggplot2

Warning message:
"package 'tidyverse' was built under R version 3.6.3"
-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v tibble  3.1.0     v purrr   0.3.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

Warning message:
"package 'tibble' was built under R version 3.6.2"
Warning message:
"package 'tidyr' was built under R version 3.6.3"
Warning message:
"package 'readr' was built under R version 3.6.3"
Warning message:
"pack

## Data partition

In [2]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [3]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    491     491 

## Models

In [4]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [5]:
model_rf <- train(x=TrainData, y=TrainClasses, 
               method = "ranger",
               trControl = ctrl) 

+ Fold01.Rep1: min.node.size= 4, mtry=49, splitrule=gini 
- Fold01.Rep1: min.node.size= 4, mtry=49, splitrule=gini 
+ Fold01.Rep1: min.node.size=14, mtry=24, splitrule=gini 
- Fold01.Rep1: min.node.size=14, mtry=24, splitrule=gini 
+ Fold01.Rep1: min.node.size=12, mtry=13, splitrule=extratrees 
- Fold01.Rep1: min.node.size=12, mtry=13, splitrule=extratrees 
+ Fold02.Rep1: min.node.size= 4, mtry=49, splitrule=gini 
- Fold02.Rep1: min.node.size= 4, mtry=49, splitrule=gini 
+ Fold02.Rep1: min.node.size=14, mtry=24, splitrule=gini 
- Fold02.Rep1: min.node.size=14, mtry=24, splitrule=gini 
+ Fold02.Rep1: min.node.size=12, mtry=13, splitrule=extratrees 
- Fold02.Rep1: min.node.size=12, mtry=13, splitrule=extratrees 
+ Fold03.Rep1: min.node.size= 4, mtry=49, splitrule=gini 
- Fold03.Rep1: min.node.size= 4, mtry=49, splitrule=gini 
+ Fold03.Rep1: min.node.size=14, mtry=24, splitrule=gini 
- Fold03.Rep1: min.node.size=14, mtry=24, splitrule=gini 
+ Fold03.Rep1: min.node.size=12, mtry=13, splitr

In [6]:
model_rrf <- train(x=TrainData, y=TrainClasses, 
               method = "RRF",
               trControl = ctrl) 

Registered S3 method overwritten by 'RRF':
  method      from        
  plot.margin randomForest



+ Fold01.Rep1: mtry=29, coefReg=0.5748, coefImp=0.5343 
- Fold01.Rep1: mtry=29, coefReg=0.5748, coefImp=0.5343 
+ Fold01.Rep1: mtry=64, coefReg=0.9006, coefImp=0.4684 
- Fold01.Rep1: mtry=64, coefReg=0.9006, coefImp=0.4684 
+ Fold01.Rep1: mtry=46, coefReg=0.2782, coefImp=0.8368 
- Fold01.Rep1: mtry=46, coefReg=0.2782, coefImp=0.8368 
+ Fold02.Rep1: mtry=29, coefReg=0.5748, coefImp=0.5343 
- Fold02.Rep1: mtry=29, coefReg=0.5748, coefImp=0.5343 
+ Fold02.Rep1: mtry=64, coefReg=0.9006, coefImp=0.4684 
- Fold02.Rep1: mtry=64, coefReg=0.9006, coefImp=0.4684 
+ Fold02.Rep1: mtry=46, coefReg=0.2782, coefImp=0.8368 
- Fold02.Rep1: mtry=46, coefReg=0.2782, coefImp=0.8368 
+ Fold03.Rep1: mtry=29, coefReg=0.5748, coefImp=0.5343 
- Fold03.Rep1: mtry=29, coefReg=0.5748, coefImp=0.5343 
+ Fold03.Rep1: mtry=64, coefReg=0.9006, coefImp=0.4684 
- Fold03.Rep1: mtry=64, coefReg=0.9006, coefImp=0.4684 
+ Fold03.Rep1: mtry=46, coefReg=0.2782, coefImp=0.8368 
- Fold03.Rep1: mtry=46, coefReg=0.2782, coefImp=

## Results

In [7]:
print(model_rf)
plot(model_rf)

Random Forest 

982 samples
 73 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 884, 884, 884, 884, 884, 884, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  splitrule   Accuracy   Kappa    
   4             49    gini        0.7729643  0.5459802
  12             13    extratrees  0.7576788  0.5153700
  14             24    gini        0.7729540  0.5459701

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 49, splitrule = gini
 and min.node.size = 4.


In [8]:
print(model_rrf)
plot(model_rrf) 

Regularized Random Forest 

982 samples
 73 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 884, 883, 883, 884, 884, 884, ... 
Resampling results across tuning parameters:

  mtry  coefReg    coefImp    Accuracy   Kappa    
  29    0.5747602  0.5342931  0.6985570  0.3970303
  46    0.2781957  0.8368328  0.7169553  0.4338109
  64    0.9006053  0.4684300  0.7362709  0.4724910

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 64, coefReg = 0.9006053
 and coefImp = 0.46843.


In [9]:
# collect resamples
results <- resamples(list(ranger=model_rf,rrf=model_rrf))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: ranger, rrf 
Number of resamples: 10 

Accuracy 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
ranger 0.7244898 0.7474747 0.7602041 0.7729643 0.7908163 0.8469388    0
rrf    0.6938776 0.7114255 0.7346939 0.7362709 0.7468306 0.7959184    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
ranger 0.4489796 0.4951040 0.5204082 0.5459802 0.5816327 0.6938776    0
rrf    0.3877551 0.4228958 0.4693878 0.4724910 0.4932355 0.5918367    0


## Prediction

In [10]:
pred_rf <- predict(model_rf,TestData)  
pred_rf.prob <- predict(model_rf,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_rf) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      96      51
   NotFast   44     230
                                          
               Accuracy : 0.7743          
                 95% CI : (0.7314, 0.8134)
    No Information Rate : 0.6675          
    P-Value [Acc > NIR] : 1.01e-06        
                                          
                  Kappa : 0.498           
                                          
 Mcnemar's Test P-Value : 0.5382          
                                          
            Sensitivity : 0.6857          
            Specificity : 0.8185          
         Pos Pred Value : 0.6531          
         Neg Pred Value : 0.8394          
             Prevalence : 0.3325          
         Detection Rate : 0.2280          
   Detection Prevalence : 0.3492          
      Balanced Accuracy : 0.7521          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.6857143            0.8185053            0.6530612 
      Neg Pred Value            Precision               Recall 
           0.8394161            0.6530612            0.6857143 
                  F1           Prevalence       Detection Rate 
           0.6689895            0.3325416            0.2280285 
Detection Prevalence    Balanced Accuracy 
           0.3491686            0.7521098

In [11]:
roc_obj <- roc(TestClasses, as.numeric(pred_rf))
plot(roc_obj, main="Curva ROC")

Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [12]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_rf))

Data: as.numeric(pred_rf) in 147 controls (TestClasses fast) < 274 cases (TestClasses NotFast).
Area under the curve: 0.7462

In [13]:
pred_rf.prob

fast,NotFast
0.2748333,0.72516667
0.9213333,0.07866667
0.4900000,0.51000000
0.3953333,0.60466667
0.5716667,0.42833333
0.5488333,0.45116667
0.3636667,0.63633333
0.4120000,0.58800000
0.3321667,0.66783333
0.0935000,0.90650000
